In [ ]:
!pip install bertopic[visualization]

In [ ]:
!pip install nbformat

In [5]:
#정제파일 불러오기
import pandas as pd
data=pd.read_csv('/content/정제파일.csv',encoding='utf-8')
column_name='제목_본문'
preprocessed_documents=data[column_name]

In [6]:
print(preprocessed_documents)

0      ['나라', '지식', '정보', '컨소시엄', '문화', '게임', '콘텐츠', ...
1      ['리사', '박사', '반도체', '코로나', '계기', '현대', '생활', '...
2      ['김포공항', '용산', '에어택시', '하늘', '에어택시', '이자', '드론...
3      ['브랜드', '론칭', '스포츠', '승부', '예측', '적용', '유플러스',...
4      ['오늘', '날씨', '오후', '월요일', '오후', '날씨', '시간', '현...
                             ...                        
294    ['내년', '정통부', '예산', '확정', '반도체', '미래', '기술', '...
295    ['문장', '음성', '인공', '지능', '목소리', '자신', '육성', '인...
296    ['인삼공사', '개인맞춤', '스케', '시장', '진출', '국내', '건강',...
297    ['대학원', '유치', '가장', '보람', '세계', '연구', '그룹', '목...
298    ['영화', '과학자', '찾기', '연구', '등장', '분석', '개발', '관...
Name: 제목_본문, Length: 299, dtype: object


In [7]:
preprocessed_documents=preprocessed_documents.values.tolist()

In [8]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP
import hdbscan

# 사전 학습된 다국어 임베딩 모델 로드
embedding_model = SentenceTransformer("sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens")
# 사용자 정의 CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1, 1))
# BERTopic 모델 생성 (각 파라미터를 데이터가 적은 상황에 맞게 조정)
model = BERTopic(
    embedding_model=embedding_model,  # 사전 학습된 다국어 임베딩 모델
    vectorizer_model=vectorizer,  # 사용자 정의 벡터화 모델 (CountVectorizer)
    nr_topics=5,#"auto",#"auto",  # 토픽 개수를 자동으로 결정
    top_n_words=8,  # 각 토픽당 상위 8개의 단어 추출
    n_gram_range=(1, 1),  # 유니그램만 사용
    min_topic_size=3,  # 각 토픽당 최소 문서 수를 줄임
    low_memory=True,  # 메모리 사용량 감소를 위해 UMAP의 low_memory를 True로 설정
    verbose=True,  # 모델의 단계 추적을 위해 True로 설정
)


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.09k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
topics, probs = model.fit_transform(preprocessed_documents)

2024-08-11 12:13:20,736 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/10 [00:00<?, ?it/s]

2024-08-11 12:15:42,856 - BERTopic - Embedding - Completed ✓
2024-08-11 12:15:42,860 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-08-11 12:15:54,902 - BERTopic - Dimensionality - Completed ✓
2024-08-11 12:15:54,905 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-08-11 12:15:54,947 - BERTopic - Cluster - Completed ✓
2024-08-11 12:15:54,950 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-08-11 12:15:55,044 - BERTopic - Representation - Completed ✓
2024-08-11 12:15:55,046 - BERTopic - Topic reduction - Reducing number of topics
2024-08-11 12:15:55,127 - BERTopic - Topic reduction - Reduced number of topics from 9 to 5


In [10]:
model.get_topic_info(0) #count의 경우 sentence 개수임. -1번 토픽은 제외하기

,Topic,Count,Name,Representation,Representative_Docs
0,0,217,0_지능_인공_기술_개발,"[지능, 인공, 기술, 개발, 교육, 미래, 기업, 산업]","[['사이버', '한국', '외대', '방송', '콘텐츠', '번역', '데이터',..."


In [11]:
m=model.get_topic_info() #count의 경우 sentence 개수임. -1번 토픽은 제외하기

In [12]:
filename='file'
m.to_csv(f'{filename}_get_topic_info.csv')

In [13]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,3,-1_팔라우_박정호_부회장_부산,"[팔라우, 박정호, 부회장, 부산, 날씨, 세계박람회, 유영상, 텔레콤]","[['오늘', '날씨', '오전', '월요일', '아침', '날씨', '시각', '..."
1,0,217,0_지능_인공_기술_개발,"[지능, 인공, 기술, 개발, 교육, 미래, 기업, 산업]","[['사이버', '한국', '외대', '방송', '콘텐츠', '번역', '데이터',..."
2,1,39,1_대통령_연구소_대구_지능,"[대통령, 연구소, 대구, 지능, 인공, 의료, 주년, 서울시]","[['쇠락', '맏형', '연구', '단지', '혁신', '창업', '전진기', '..."
3,2,26,2_날씨_기온_수원_인천,"[날씨, 기온, 수원, 인천, 아침, 서울, 춘천, 전주]","[['오늘', '날씨', '오전', '토요일', '아침', '날씨', '아침', '..."
4,3,14,3_지방_날씨_현재_오후,"[지방, 날씨, 현재, 오후, 대부분, 구름, 맑은, 서울]","[['오늘', '날씨', '오후', '일요일', '오후', '날씨', '시간', '..."


In [14]:
model.visualize_heatmap()

In [15]:
model.visualize_barchart(top_n_topics=25,n_words=8,title="Topic Scores",width=300,height=500)

In [16]:

# Visualization
# PLDAVIS와 비슷한 형태로 시각호를 지원해준다.
model.visualize_topics()